# Introduction

Nama : Nindia Ekasuci Larasati

Batch : HCK - 030

**Objektif**:

Great Expectations (GX) bertujuan untuk memastikan bahwa data yang digunakan memiliki kualitas yang baik, konsisten, dan dapat dipercaya. GX bekerja dengan melakukan validasi otomatis terhadap data, seperti mengecek format, nilai yang hilang, maupun anomali, sehingga potensi kesalahan dapat terdeteksi lebih awal sebelum data dipakai lebih lanjut. Dengan demikian, GX membantu tim data menjaga keandalan pipeline, meningkatkan akurasi hasil analisis, serta mengurangi risiko pengambilan keputusan yang salah akibat data yang tidak valid.

# Import Libraries

In [ ]:
import pandas as pd
from great_expectations.data_context import get_context

# Data Loading

Dilakukan data loading dengan data csv hasil dari proses ETL dengan menggunakan ElasticSearch. Sebelum masuk ke expetations perlu dilakukan pengecekan data ulang untuk melihat apakah format data sudah benar atau belum. 

In [2]:
# Load dataset
# Memakai parse_dates agar kolom 'order_date' langsung dibaca sebagai datetime
df = pd.read_csv(r"C:\Users\User\New_folder\P2\p2-ftds030-hck-m3-NindiaEka\dags\P2M3_nindia_ekasuci_data_transformed.csv", parse_dates=['order_date'])

In [3]:
df.info()
df.nunique()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1894 entries, 0 to 1893
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   order_id          1894 non-null   int64         
 1   customer_name     1894 non-null   object        
 2   product_category  1894 non-null   object        
 3   product_name      1894 non-null   object        
 4   units_sold        1894 non-null   float64       
 5   unit_price        1894 non-null   float64       
 6   discount          1894 non-null   float64       
 7   sales_amount      1894 non-null   float64       
 8   order_date        1894 non-null   datetime64[ns]
 9   city              1894 non-null   object        
 10  segment           1894 non-null   object        
 11  profit            1894 non-null   float64       
 12  unique_id         1894 non-null   object        
dtypes: datetime64[ns](1), float64(5), int64(1), object(6)
memory usage: 192.5+ KB


order_id            1819
customer_name       1868
product_category       6
product_name          24
units_sold             8
unit_price           990
discount              72
sales_amount         143
order_date           561
city                   9
segment                2
profit              1889
unique_id           1894
dtype: int64

# Setup Datasource & Batch Request

Pada tahap ini dilakukan proses inisialisasi Datasource dan Data Asset di Great Expectations. Pertama, ditentukan nama unik untuk datasource, yaitu sales__clothes, agar dapat dibedakan dengan datasource lain di dalam proyek. Selanjutnya, dibuat GX DataContext untuk mengelola konfigurasi dan sumber data. Datasource kemudian ditambahkan dengan menggunakan metode add_pandas, karena data akan diakses melalui Pandas DataFrame.

Kemudian didefinisikan sebuah data asset dengan nama sales, yang mengambil sumber dari file CSV hasil transformasi (P2M3_nindia_ekasuci_data_transformed.csv). Data asset ini berfungsi sebagai representasi dataset yang akan divalidasi.

Tahap terakhir adalah membangun batch request dari data asset tersebut, yang nantinya digunakan Validator untuk menjalankan serangkaian expectation dalam proses validasi data.

In [5]:
#Give a name to a Datasource. This name must be unique between Datasources.
datasource_name = 'sales__clothes'

# Ambil GX DataContext
context = get_context()
datasource = context.sources.add_pandas(datasource_name)

# Give a name to a data asset
asset_name = 'sales'
path_to_data = r"C:\Users\User\New_folder\P2\p2-ftds030-hck-m3-NindiaEka\dags\P2M3_nindia_ekasuci_data_transformed.csv"
asset = datasource.add_csv_asset(asset_name, filepath_or_buffer=path_to_data)

# Build batch request
batch_request = asset.build_batch_request()

# Membuat Expectation Suite

Expectation Suite adalah kumpulan aturan atau pernyataan yang dapat diuji untuk memvalidasi kualitas data. Sebuah Expectation Suite menyatukan berbagai Expectation sehingga membentuk gambaran menyeluruh tentang kondisi data. Penamaan Expectation Suite dapat ditentukan secara bebas, asalkan unik di dalam satu proyek. Untuk membuat dan menguji Expectation Suite, digunakan Validator yang berfungsi mengakses batch data. Setiap kali sebuah Expectation dijalankan melalui validator.expect_*, hasil validasinya langsung diterapkan pada data yang diuji.

In [6]:
# Creat an expectation suite
expectation_suite_name = 'expectation-sales-clothes'
context.add_or_update_expectation_suite(expectation_suite_name)

# Create a validator using above expectation suite
validator = context.get_validator(
    batch_request = batch_request,
    expectation_suite_name = expectation_suite_name
)

# Check the validator
validator.head()

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

,order_id,customer_name,product_category,product_name,units_sold,unit_price,discount,sales_amount,order_date,city,segment,profit,unique_id
0,1000,Brian Thompson,Jeans,Slim Fit Jeans,2.655779,842.000000,0.600000,0.0,2025-02-27,Delhi,B2C,2137.45,1000_Brian Thompson
1,1001,Shaun Ross,Jeans,Slim Fit Jeans,1.000000,2691.715527,0.646466,0.0,2025-07-15,Ahmedabad,B2C,1588.15,1001_Shaun Ross
2,1002,Sarah Snyder,Jackets,Puffer Coat,1.000000,637.820000,0.646466,0.0,2025-01-02,Mumbai,B2B,-158.03,1002_Sarah Snyder
3,1003,Jay Briggs,Shoes,Loafers,2.000000,2962.270000,0.646466,0.0,2025-06-18,bengaluru,B2B,2296.50,1003_Jay Briggs
4,1005,Samuel Miller,T-Shirts,Crop Top,2.655779,2691.715527,0.646466,0.0,2023-12-05,Mumbai,B2B,1477.73,1005_Samuel Miller


# Expetations

## Expetations 1 : to be unique

 <div align="justify">
 
Kolom unique_id dibentuk dari gabungan order_id dan customer_name untuk memastikan setiap transaksi benar-benar unik. Hal ini dilakukan karena pada sistem transaksi e-commerce sering ditemukan kasus di mana satu order_id dapat muncul berulang akibat retur barang atau pencatatan ulang dengan nama pelanggan yang sama. Dengan adanya kolom ini, proses validasi data menjadi lebih akurat karena potensi duplikasi transaksi dapat dihindari. Selain itu, unique_id juga bermanfaat dalam supply chain dan logistik untuk melacak distribusi barang secara spesifik, serta mendukung analisis pelanggan dan forecasting permintaan yang lebih tepat karena data transaksi sudah terjamin keunikannya.

In [7]:
validator.expect_column_values_to_be_unique("unique_id")

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 1894,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

## Expetations 2 : Discount harus bernilai 0 - 0.7

 <div align="justify">
 
Diskon harus berupa persentase valid, sehingga tidak boleh negatif atau lebih dari 100%. Sebelum dilakukan proses validasi ternyata pada kolom discount masih memiliki nilai yang tidak masuk akal. Yaitu ada nilai diatas 1(1.25, 1.14 dll). Dimana jika dirubah kedalam persenan yaitu bernilai 100% lebih. Tidak mungkin ada diskon yang bernilai diatas 100%. Dan masih ada banyak diskon yang tidak wajar seperti 0.99, 0.80 dll. Maka dari itu agar nilai pada kolom discount masuk akal dilakukan handling outlier pada preporcess data menggunakan metode clip. Metode ini mempertahankan data tapi mengepress data ke batas maksimumnya. Dimana pada hal ini, batas wajar dalam pemberian diskon adalah 0.7 (70%).

In [8]:
validator.expect_column_values_to_be_between("discount", min_value=0, max_value=0.7)

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 1894,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

## Expetations 3 : Segment tidak boleh memiliki nilai null

Expectation ini digunakan untuk memvalidasi bahwa kolom segment tidak mengandung nilai kosong (null atau missing value). Apabila terdapat nilai kosong di kolom ini, maka analisis yang berbasis segmentasi pelanggan akan menjadi tidak akurat dan berpotensi menimbulkan kesalahan interpretasi dalam proses pengambilan keputusan, khususnya dalam supply chain maupun strategi pemasaran.

In [9]:
validator.expect_column_values_to_not_be_null('segment')

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 1894,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": []
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

## Expetations 4 : product_category hanya boleh berisi kategori valid (Jeans, Jackets, Shoes, T-Shirts, Accessories, Dresses).

Kolom kategori produk tidak boleh mengandung nilai aneh atau typo. Hanya kategori yang ada di sistem (Jeans, Jackets, Shoes, T-Shirts, Accessories, Dresses)

In [10]:
validator.expect_column_values_to_be_in_set(
    "product_category",
    ['Jeans', 'Jackets', 'Shoes', 'T-Shirts', 'Accessories', 'Dresses']
)

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 1894,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

Hasil pengujian menunjukkan bahwa seluruh data pada kolom segment memenuhi kriteria, dengan total 1.894 baris data tervalidasi dan tidak ditemukan satupun nilai null. Indikator ini ditunjukkan oleh unexpected_count = 0 dan unexpected_percent = 0.0%. Dengan demikian, dapat disimpulkan bahwa dataset memiliki kualitas yang baik dari sisi kelengkapan informasi pada kolom segment.

## Expectation 5: Validasi tipe data kolom order_date sebagai datetime

Expectation ini digunakan untuk memverifikasi bahwa kolom order_date benar-benar berisi data dengan format tanggal yang valid. Validasi ini penting karena kolom order_date menjadi dasar dalam analisis tren penjualan, forecasting permintaan, serta evaluasi performa supply chain berdasarkan periode waktu tertentu. Jika format tanggal tidak konsisten atau terdapat nilai yang salah, maka hasil analisis berbasis waktu dapat terganggu dan menurunkan akurasi prediksi.

In [11]:
validator.expect_column_values_to_match_strftime_format("order_date", "%Y-%m-%d")

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 1894,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

Pengujian dilakukan dengan menggunakan expectation expect_column_values_to_match_strftime_format, di mana setiap nilai pada kolom order_date diperiksa kesesuaiannya dengan pola format tanggal %Y-%m-%d (contoh: 2023-08-15). Hasil pengujian menunjukkan bahwa seluruh 1.894 baris data memenuhi kriteria, dengan nilai unexpected_count = 0 dan unexpected_percent = 0.0%. Tidak ada satupun nilai yang teridentifikasi sebagai error, null, atau tidak sesuai format.

## Expectation 6: Validasi Nilai Distinct pada Kolom Segment

Validasi dilakukan pada kolom segment menggunakan expectation expect_column_distinct_values_to_equal_set.
Expectation ini bertujuan memastikan bahwa seluruh nilai unik dalam kolom tersebut hanya terdiri dari kategori yang sudah didefinisikan, yaitu: B2C dan B2B. 


In [12]:
validator.expect_column_distinct_values_to_equal_set(
    "segment",
    value_set=["B2C", "B2B"]
)


Calculating Metrics:   0%|          | 0/4 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "observed_value": [
      "B2B",
      "B2C"
    ],
    "details": {
      "value_counts": [
        {
          "value": "B2B",
          "count": 630
        },
        {
          "value": "B2C",
          "count": 1264
        }
      ]
    }
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

Hasil validasi menunjukkan bahwa nilai distinct pada kolom segment memang sesuai dengan set yang diharapkan, sehingga dapat disimpulkan bahwa tidak terdapat kategori di luar daftar yang valid. Dengan demikian, kolom segment memenuhi standar kualitas data yang telah ditentukan.

## Expetations 7 : Profit tidak boleh negatif terlalu besar

Expectation ini digunakan untuk memverifikasi bahwa struktur tabel sesuai dengan skema yang telah ditentukan, baik dari sisi nama kolom maupun urutannya. Validasi dilakukan dengan menggunakan expect_table_columns_to_match_ordered_list, di mana daftar kolom yang diharapkan terdiri dari:

- order_id

- customer_name

- product_category

- product_name

- units_sold

- unit_price

- discount

- sales_amount

- order_date

In [13]:
validator.expect_table_columns_to_match_ordered_list(
    column_list=["order_id", "customer_name", "product_category",	"product_name",	"units_sold",	"unit_price",	"discount",	"sales_amount",	"order_date",	"city",	"segment",	"profit",	"unique_id"]
)


Calculating Metrics:   0%|          | 0/2 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "observed_value": [
      "order_id",
      "customer_name",
      "product_category",
      "product_name",
      "units_sold",
      "unit_price",
      "discount",
      "sales_amount",
      "order_date",
      "city",
      "segment",
      "profit",
      "unique_id"
    ]
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

Hasil pengujian menunjukkan bahwa seluruh kolom dalam tabel sesuai dengan urutan yang diharapkan (success: true). Dari output yang diperoleh, observed_value sama persis dengan column_list, yang berarti tidak ada kolom yang hilang, berlebih, maupun salah urut.

In [14]:
# Save suite ke context
validator.save_expectation_suite(discard_failed_expectations=False)

# Checkpoints

Checkpoint pada Great Expectations digunakan untuk menjalankan seluruh Expectation Suite terhadap dataset secara otomatis dan terdokumentasi. Pada tahap ini dibuat checkpoint bernama checkpoint_1 yang mengacu pada suite expectation-sales-clothes, lalu dijalankan untuk mengevaluasi semua aturan validasi. 

In [15]:
# Create a checkpoint

checkpoint_1 = context.add_or_update_checkpoint(
    name="checkpoint_1",
    validator=validator,
    expectation_suite_name="expectation-sales-clothes"
)

In [16]:
# Run checkpoint
checkpoint_result = checkpoint_1.run()

Calculating Metrics:   0%|          | 0/37 [00:00<?, ?it/s]

Hasilnya menunjukkan bahwa seluruh 37 expectation berhasil dijalankan dan lolos validasi, sehingga dapat disimpulkan bahwa dataset sudah memenuhi standar kualitas yang ditetapkan dan siap digunakan untuk analisis maupun kebutuhan bisnis selanjutnya.